# High-level TF Example

In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
from common.params import *
from common.utils import *

In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.1
Tensorflow:  1.2.1


In [3]:
def create_symbol():
    conv1 = tf.layers.conv2d(X, filters=50, kernel_size=(3, 3), padding='same')
    relu1 = tf.nn.relu(conv1)
    conv2 = tf.layers.conv2d(relu1, filters=50, kernel_size=(3, 3), padding='same')
    relu2 = tf.nn.relu(conv2)
    pool1 = tf.layers.max_pooling2d(relu2, pool_size=(2, 2), strides=(2, 2), padding='valid')
    drop1 = tf.layers.dropout(pool1, 0.25)
    
    conv3 = tf.layers.conv2d(drop1, filters=100, kernel_size=(3, 3), padding='same')
    relu3 = tf.nn.relu(conv3)
    conv4 = tf.layers.conv2d(relu3, filters=100, kernel_size=(3, 3), padding='same')
    relu4 = tf.nn.relu(conv4)
    pool2 = tf.layers.max_pooling2d(relu4, pool_size=(2, 2), strides=(2, 2), padding='valid')
    drop2 = tf.layers.dropout(pool2, 0.25)
    
    flatten = tf.reshape(drop2, shape=[-1, 100*8*8])
    fc1 = tf.layers.dense(flatten, 512, activation=tf.nn.relu)
    drop4 = tf.layers.dropout(fc1, 0.5)
    logits = tf.layers.dense(drop4, N_CLASSES, name='output')
    return logits

In [4]:
def init_model(m):
    # Single-class labels, don't need dense one-hot
    # Expects unscaled logits, not output of tf.nn.softmax
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=m, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.MomentumOptimizer(learning_rate=LR, momentum=MOMENTUM)
    training_op = optimizer.minimize(loss)
    return training_op

In [5]:
%%time
# Data into format for library
#x_train, x_test, y_train, y_test = mnist_for_library(channel_first=False)
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=False)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Preparing train set...
Preparing test set...
Done.
(50000, 32, 32, 3) (10000, 32, 32, 3) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 3.12 s, sys: 1.46 s, total: 4.58 s
Wall time: 26 s


In [6]:
%%time
# Place-holders
X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.int32, shape=[None])
# Initialise model
sym = create_symbol()

CPU times: user 88 ms, sys: 0 ns, total: 88 ms
Wall time: 88 ms


In [7]:
%%time
model = init_model(sym)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# Accuracy logging
correct = tf.nn.in_top_k(sym, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

CPU times: user 316 ms, sys: 396 ms, total: 712 ms
Wall time: 730 ms


In [8]:
%%time
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        sess.run(model, feed_dict={X: data, y: label})
    # Log
    acc_train = sess.run(accuracy, feed_dict={X: data, y: label})
    print(j, "Train accuracy:", acc_train)

0 Train accuracy: 0.59375
1 Train accuracy: 0.59375
2 Train accuracy: 0.8125
3 Train accuracy: 0.828125
4 Train accuracy: 0.875
5 Train accuracy: 0.90625
6 Train accuracy: 1.0
7 Train accuracy: 0.984375
8 Train accuracy: 0.984375
9 Train accuracy: 1.0
CPU times: user 3min 10s, sys: 40.7 s, total: 3min 51s
Wall time: 5min


In [9]:
%%time
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    pred = tf.argmax(sym,1)
    output = sess.run(pred, feed_dict={X: data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 3 s, sys: 348 ms, total: 3.35 s
Wall time: 3.24 s


In [10]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.721354166667
